In [2]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다"""

### 단어 분리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 단어 분리
t = Tokenizer()
t.fit_on_texts([text])

t.word_docs

defaultdict(int,
            {'뛰고': 1,
             '고와야': 1,
             '곱다': 1,
             '경마장에': 1,
             '있다': 1,
             '법이다': 1,
             '가는': 1,
             '있는': 1,
             '오는': 1,
             '그의': 1,
             '말이': 1})

In [4]:
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [5]:
# 단어와 인덱스 출력
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


### 토큰화와 정수 인코딩

In [6]:
# 훈련 데이터 생성
sequences = []

# Wn을 기준으로 문장 토큰화
for line in text.split('\n'): 
  
    # 단어를 정수로 변경
    encoded = t.texts_to_sequences([line])[0]
    
    # 2개 이상의 모든 단의 조합을 생성
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [7]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


- 위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터

- [2, 3] 은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당

- 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 함

- 우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준

- 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6

In [8]:
print(max(len(l) for l in sequences)) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력

6


### 입력할 내용을 일정한 길이로 변경

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 6

# pad_sequences() : 모든 샘플에 대해서 0을 사용하여 길이를 맞춤 
# pre : 길이가 6보다 짧은 샘플의 앞에 0으로 채움
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [10]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


### 특성과 라벨을 분리

In [10]:
import numpy as np

# 마지막 단어를 레이블로 분리
sequences = np.array(sequences)

X = sequences[:,:-1]
y = sequences[:,-1]

In [11]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [12]:
print(y) 

[ 3  1  4  5  1  7  1  9 10  1 11]


### 원 핫 인코딩 

In [13]:
from tensorflow.keras.utils import to_categorical

# 첫 번째 인덱스는 사용하지 않음
y_en = to_categorical(y, num_classes=vocab_size)

In [14]:
print(y_en)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [16]:
X

array([[ 0,  0,  0,  0,  2],
       [ 0,  0,  0,  2,  3],
       [ 0,  0,  2,  3,  1],
       [ 0,  2,  3,  1,  4],
       [ 0,  0,  0,  0,  6],
       [ 0,  0,  0,  6,  1],
       [ 0,  0,  0,  0,  8],
       [ 0,  0,  0,  8,  1],
       [ 0,  0,  8,  1,  9],
       [ 0,  8,  1,  9, 10],
       [ 8,  1,  9, 10,  1]])

ERROR! Session/line number was not unique in database. History logging moved to new session 758


### 신경망 설계 

In [18]:
X

array([[ 0,  0,  0,  0,  2],
       [ 0,  0,  0,  2,  3],
       [ 0,  0,  2,  3,  1],
       [ 0,  2,  3,  1,  4],
       [ 0,  0,  0,  0,  6],
       [ 0,  0,  0,  6,  1],
       [ 0,  0,  0,  0,  8],
       [ 0,  0,  0,  8,  1],
       [ 0,  0,  8,  1,  9],
       [ 0,  8,  1,  9, 10],
       [ 8,  1,  9, 10,  1]])

In [1]:
vocab_size

NameError: name 'vocab_size' is not defined

In [15]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.models import Sequential

model = Sequential()
# 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) 
model.add(SimpleRNN(32)) #분석
model.add(Dense(vocab_size, activation='softmax')) #판단

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y_en, epochs=200, verbose=2)

Train on 11 samples
Epoch 1/200
11/11 - 1s - loss: 2.4687 - accuracy: 0.0909
Epoch 2/200
11/11 - 0s - loss: 2.4556 - accuracy: 0.1818
Epoch 3/200
11/11 - 0s - loss: 2.4425 - accuracy: 0.3636
Epoch 4/200
11/11 - 0s - loss: 2.4292 - accuracy: 0.3636
Epoch 5/200
11/11 - 0s - loss: 2.4155 - accuracy: 0.3636
Epoch 6/200
11/11 - 0s - loss: 2.4015 - accuracy: 0.3636
Epoch 7/200
11/11 - 0s - loss: 2.3870 - accuracy: 0.3636
Epoch 8/200
11/11 - 0s - loss: 2.3719 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.3562 - accuracy: 0.4545
Epoch 10/200
11/11 - 0s - loss: 2.3398 - accuracy: 0.4545
Epoch 11/200
11/11 - 0s - loss: 2.3228 - accuracy: 0.4545
Epoch 12/200
11/11 - 0s - loss: 2.3049 - accuracy: 0.4545
Epoch 13/200
11/11 - 0s - loss: 2.2864 - accuracy: 0.5455
Epoch 14/200
11/11 - 0s - loss: 2.2669 - accuracy: 0.5455
Epoch 15/200
11/11 - 0s - loss: 2.2467 - accuracy: 0.5455
Epoch 16/200
11/11 - 0s - loss: 2.2256 - accuracy: 0.4545
Epoch 17/200
11/11 - 0s - loss: 2.2036 - accuracy: 0.4545
Epo

Epoch 142/200
11/11 - 0s - loss: 0.3271 - accuracy: 0.9091
Epoch 143/200
11/11 - 0s - loss: 0.3219 - accuracy: 0.9091
Epoch 144/200
11/11 - 0s - loss: 0.3168 - accuracy: 0.9091
Epoch 145/200
11/11 - 0s - loss: 0.3118 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.3068 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.3019 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2971 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2924 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2877 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2831 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2785 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2740 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.2696 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.2652 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.2609 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.2566 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.2524 - accuracy: 1.00

## 문장을 생성하는 sentence_generation() 함수

In [22]:
t.word_index.items()

dict_items([('말이', 1), ('경마장에', 2), ('있는', 3), ('뛰고', 4), ('있다', 5), ('그의', 6), ('법이다', 7), ('가는', 8), ('고와야', 9), ('오는', 10), ('곱다', 11)])

In [23]:
# 사용할 모델, 토크나이저, 현재 단어, 생성할 단어 수
def sentence_generation(model, t, current_word, n): 
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        # 현재 단어에 대한 정수 인코딩
        encoded = t.texts_to_sequences([current_word])[0] 
        print(encoded)
        # 데이터를 같은 길이로 맞춤 (pre : 앞 부분에 0을 추가, post : 뒷 부분에 0을 추가)
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') 
        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict_classes(encoded, verbose=0)        
        
        for word, index in t.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            # 해당 단어가 예측 단어이므로 break
            if index == result: 
                break 
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경        
        current_word = current_word + ' '  + word 
        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word 

    # 문장 생성
    sentence = init_word + sentence
    
    return sentence

### 임의의 단어로 시작하는 설정 개수만큼의 단어로 문장을 생성

In [29]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측 가능

[2]
[2, 3]
[2, 3, 1]
[2, 3, 1, 4]
경마장에 있는 말이 뛰고 있다


In [30]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

[6]
[6, 1]
그의 말이 법이다


In [31]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

[8]
[8, 1]
[8, 1, 9]
[8, 1, 9, 10]
[8, 1, 9, 10, 1]
가는 말이 고와야 오는 말이 곱다


## LSTM을 이용하여 텍스트 생성 (더 많은 데이터 활용)

- 뉴욕 타임즈 기사 다운로드 : https://www.kaggle.com/aashita/nyt-comments

- 빅카인즈 뉴스기사 데이터 활용(https://www.bigkinds.or.kr/)

(1) 검색키워드 입력

(2) 하단의 "뉴스분석" 버튼 클릭

(3) 하단의 "다운로드" 버튼 클릭

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 460kB 48.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt

# "미국 대선"으로 검색한 데이터
news = pd.read_excel("/gdrive/My Drive/Colab Notebooks/인공지능강의안/data/news_data.xlsx")

In [6]:
news['제목'] = news['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","") 

headline = []

okt = Okt()

stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와', '등', '으로도']

for sentence in news['제목']:
    temp_X = []
    temp_X=okt.morphs(sentence, stem=True) 
    temp_X=[word for word in temp_X if not word in stopwords] 
    headline.append(temp_X)

headline[:5]    

[['바이든', '대', '주주', '요건', '완화', '활짝', '웃다', '코스피', '앞', '쭉'],
 ['강경화', '대선', '후', '미국', '행', '이인영', '방미', '추진'],
 ['백악관', '주변', '대형', '울타리', '설치', '미국', '대선', '긴장', '고조'],
 ['역사상', '가장', '살벌하다', '선거', '워싱턴', '곳곳', '주', '방위', '군', '배치'],
 ['경합주', '박빙', '백인', '블루', '칼라', '사커맘', '표심', '변수']]

In [7]:
from keras.preprocessing.text import Tokenizer

max_feature = 5000
tokenizer = Tokenizer(num_words=max_feature) 
# 단어에 인덱스를 부여하기 위한 분석
tokenizer.fit_on_texts(headline)

vocab_size = len(tokenizer.word_index) + 1

# 문자열을 정수 인덱스의 리스트로 변환
headline_en = tokenizer.texts_to_sequences(headline)

headline_en[:5]  

[[3, 53, 178, 301, 302, 303, 179, 42, 180, 304],
 [87, 1, 305, 4, 181, 27, 54, 55],
 [115, 306, 307, 308, 182, 4, 1, 183, 309],
 [184, 116, 310, 23, 311, 117, 28, 70, 56, 118],
 [15, 119, 312, 313, 314, 315, 316, 317]]

In [8]:
sequences = list()

# 샘플을 1개씩 가져온다.
for line in headline_en: 
    # 각 단어들로 구성된 모든 문장 조합을 생성
    for i in range(1, len(line)):
        sequence = line[:i+1]
        sequences.append(sequence)

# 11개의 샘플 출력
sequences[:11] 

[[3, 53],
 [3, 53, 178],
 [3, 53, 178, 301],
 [3, 53, 178, 301, 302],
 [3, 53, 178, 301, 302, 303],
 [3, 53, 178, 301, 302, 303, 179],
 [3, 53, 178, 301, 302, 303, 179, 42],
 [3, 53, 178, 301, 302, 303, 179, 42, 180],
 [3, 53, 178, 301, 302, 303, 179, 42, 180, 304],
 [87, 1],
 [87, 1, 305]]

In [9]:
index_to_word={}

# 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
for key, value in tokenizer.word_index.items(): 
    index_to_word[value] = key

index_to_word[100]

'믿다'

In [ ]:
tokenizer.word_index

In [13]:
# 문장의 최대 길이
max_len=max(len(s) for s in sequences)
print(max_len)

19


### (실습) 입력할 내용을 일정한 길이로 변경

- maxlen=max_len로 설정

In [14]:
from keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   3
   53]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   3  53
  178]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   3  53 178
  301]]


### (실습) 특성과 라벨을 분리

In [15]:
import numpy as np

sequences = np.array(sequences)

X = sequences[:,:-1]
y = sequences[:,-1]

In [21]:
print(X[:4])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   3  53]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   3  53 178]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   3  53 178 301]]


In [20]:
print(y[:3]) # 레이블

[ 53 178 301]


### (실습) 원 핫 인코딩

In [22]:
from keras.utils import to_categorical

y = to_categorical(y, num_classes=vocab_size)

### (실습) 신경망 설계 (Embedding, LSTM/GRU) / complie / fit 수행

In [23]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential

model = Sequential()
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs=200)

Epoch 1/200
61/61 [==============================] - 34s 10ms/step - loss: 6.5983 - accuracy: 0.0237
Epoch 2/200
61/61 [==============================] - 0s 5ms/step - loss: 6.0702 - accuracy: 0.0377
Epoch 3/200
61/61 [==============================] - 0s 5ms/step - loss: 6.0320 - accuracy: 0.0359
Epoch 4/200
61/61 [==============================] - 0s 5ms/step - loss: 6.0002 - accuracy: 0.0410
Epoch 5/200
61/61 [==============================] - 0s 5ms/step - loss: 5.9883 - accuracy: 0.0414
Epoch 6/200
61/61 [==============================] - 0s 5ms/step - loss: 5.8843 - accuracy: 0.0431
Epoch 7/200
61/61 [==============================] - 0s 5ms/step - loss: 5.8458 - accuracy: 0.0456
Epoch 8/200
61/61 [==============================] - 0s 5ms/step - loss: 5.7690 - accuracy: 0.0442
Epoch 9/200
61/61 [==============================] - 0s 5ms/step - loss: 5.6996 - accuracy: 0.0419
Epoch 10/200
61/61 [==============================] - 0s 5ms/step - loss: 5.5351 - accuracy: 0.0507
Epoch 1

### (실습) 문장을 생성하는 sentence_generation() 함수


In [45]:
# 사용할 모델, 토크나이저, 현재 단어, 생성할 단어 수
def sentence_generation(model, t, current_word, n): 
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        # 현재 단어에 대한 정수 인코딩
        encoded = t.texts_to_sequences([current_word])[0] 
        #print(encoded)
        # 데이터를 같은 길이로 맞춤 (pre : 앞 부분에 0을 추가, post : 뒷 부분에 0을 추가)
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') 
        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict_classes(encoded, verbose=0)        
        
        for word, index in t.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            # 해당 단어가 예측 단어이므로 break
            if index == result: 
                break 
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경        
        current_word = current_word + ' '  + word 
        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word 

    # 문장 생성
    sentence = init_word + sentence
    
    return sentence

### (실습)

In [46]:
print(sentence_generation(model, tokenizer, '미 대선', 10))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


미 대선 투표 율 사상 최대 되다 것 사전투표 참여 1억 명


### (실습) 

In [47]:
print(sentence_generation(model, tokenizer, '대주주 과세', 10))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


대주주 과세 뉴스 우편 투표 담기다 대선 결과 트럼프 바이든 승자 승자
